In [1]:
import numpy as np
import pandas as pd
import os
import sys
import metrics as mt

In [2]:
test_set_path = '../sets/test_set.csv'
sub_path = '../subs/'
test_set = pd.read_csv(test_set_path, sep=',',index_col=0)
test_set = test_set['recommended_items'].apply(lambda x: pd.Series([int(i) for i in str(x).split(' ')] if x!='' else []))

In [3]:
sub_name = 'top_pop.csv'

submission = pd.read_csv(os.path.join(sub_path, sub_name), sep=',',index_col=0)
submission = submission.fillna('')
submission.columns = ['items']
sub_ranks = submission['items'].apply(lambda x: pd.Series([int(i) for i in str(x).split(' ')] if x!='' else []))

In [4]:
print('Running evaluation over {} users'.format(len(test_set)))
print('Your submission contains predictions for {} of them'.format(submission.shape[0]))
metrics = mt.compute_metrics(sub_ranks,test_set,k=5)
print('Average Recall: '), metrics[0]
print('Mean average precision at 5: '), metrics[2]
print('Average user success: '), metrics[1]

Running evaluation over 4000 users
Your submission contains predictions for 4000 of them
Average Recall:  0.00365
Mean average precision at 5:  0.0009775
Average user success:  0.01775


In [6]:
print('Competition score: '), mt.challenge_score(sub_ranks,test_set,k=5)

 Competition score:  2185.16666667
